### 1. Import Numpy, Pandas and  Redex alias them to `np` and `pd` respectively.

In [1]:
import pandas as pd
import numpy as np
import re

### 2. We are going to import out dataframe.

In [2]:
df = pd.read_csv("data/attacks.csv",encoding = "ISO-8859-1") #To load the data csv

### 3. First we want to take a look at out data

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.keys()

In [ ]:
df["Unnamed: 22"].unique()
df["Unnamed: 23"].unique()
df["href formula"].unique()
df["Case Number.1"].unique()
df["Case Number.2"].unique()
df["original order"].unique()
df["Investigator or Source"].unique()

After looking at the unique values of some of the columns in out initial dataframe, we have seen that there are some colums that does not provide us importan information to find conclusions, so we are going do delate them from our dataframe:

In [3]:
df.drop(["href", "Unnamed: 22", "Unnamed: 23", "href formula", "pdf", "Case Number.1", "Case Number.2", "original order", "Investigator or Source" ], axis=1, inplace=True)

In [ ]:
df.shape

We are going to drop all of the dataset duplicate values and save out new dateframe in df2:

In [4]:
df2 = df.drop_duplicates()

In [5]:
df2.shape

(6305, 15)

In [ ]:
df2.keys()

In [ ]:
df2.head()

In [ ]:
df2.info()

Now we are going to rename some of the dataset columns as there are some spaces within them

In [6]:
columns = list(df2.columns)
columns

['Case Number',
 'Date',
 'Year',
 'Type',
 'Country',
 'Area',
 'Location',
 'Activity',
 'Name',
 'Sex ',
 'Age',
 'Injury',
 'Fatal (Y/N)',
 'Time',
 'Species ']

In [7]:
new_columns = {column: column.replace(" ", "") for column in columns}
new_columns
df2.rename(columns = new_columns, inplace=True)
df2.head()

/Users/franciscovispo/opt/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,CaseNumber,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal(Y/N),Time,Species
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m"


# 4. Cleaning Variables

## 4.1 Through str.methods( ):

#### 4.1.1 Cleaning variable "SEX"

In [ ]:
list(df2["Sex"].unique()) # usar replace para agrupar en 3 columnas: M, F, Unkown

In [8]:
df2["Sex"] = df2["Sex"].str.replace("lli", "Unknown")
df2["Sex"] = df2["Sex"].str.replace(".", "Unknown")
df2["Sex"] = df2["Sex"].str.replace("N", "Unknown")
df2["Sex"] = df2["Sex"].str.replace("M ", "M")

list(df2["Sex"].unique())

<ipython-input-8-ab0fb109cd03>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Sex"] = df2["Sex"].str.replace("lli", "Unknown")
<ipython-input-8-ab0fb109cd03>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2["Sex"] = df2["Sex"].str.replace(".", "Unknown")
<ipython-input-8-ab0fb109cd03>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Sex

['F', 'M', nan, 'Unknown']

#### 4.1.2 Cleaning variable "TYPE"

In [9]:
# we assume that this two variables are referring to a boat, so we replace the name to sum up all the values in Boat.
df2["Type"] = df2["Type"].str.replace("Boating", "Boat")
df2["Type"] = df2["Type"].str.replace("Boatomg", "Boat")

# we see that we can sum up also all the variables in 4 general ones: unprovoked, provoked, questionable and invalid:
df2["Type"] = df2["Type"].str.replace("Boat", "Provoked")
df2["Type"] = df2["Type"].str.replace("Sea Disaster", "Unprovoked")

list(df2["Type"].unique())


<ipython-input-9-369339018a8a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Type"] = df2["Type"].str.replace("Boating", "Boat")
<ipython-input-9-369339018a8a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Type"] = df2["Type"].str.replace("Boatomg", "Boat")
<ipython-input-9-369339018a8a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

['Provoked', 'Unprovoked', 'Invalid', 'Questionable', nan]

#### 4.1.3 Cleaning variable "FATAL Y/N"

In [10]:

df2["Fatal(Y/N)"] = df2["Fatal(Y/N)"].str.replace(" N", "N")
df2["Fatal(Y/N)"] = df2["Fatal(Y/N)"].str.replace("N ", "N")
df2["Fatal(Y/N)"] = df2["Fatal(Y/N)"].str.replace("y", "Y")
df2["Fatal(Y/N)"] = df2["Fatal(Y/N)"].str.replace("M", "UNKNOWN")
df2["Fatal(Y/N)"] = df2["Fatal(Y/N)"].str.replace("2017", "UNKNOWN")
list(df2["Fatal(Y/N)"].unique())

<ipython-input-10-3b067e921e10>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Fatal(Y/N)"] = df2["Fatal(Y/N)"].str.replace(" N", "N")
<ipython-input-10-3b067e921e10>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Fatal(Y/N)"] = df2["Fatal(Y/N)"].str.replace("N ", "N")
<ipython-input-10-3b067e921e10>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

['N', 'Y', nan, 'UNKNOWN']

## 4.2 Through Regrex :

In [ ]:
# CON REGREX QUIERO INTENTAR LIMPIAR ACTIVITY Y AGE

In [ ]:
list(df2["Activity"].unique())

In [ ]:
age_values = list(df2["Age"].unique())

In [ ]:
pattern = r"\d+"


In [ ]:
for i in age_values:
    age2 = re.findall(pattern, i)
print(age2)

In [ ]:
df2["Year"] = df2["Year"].astype("Int64")
df2.Year.isna()

In [11]:
years = list(df2["Year"].unique())
type(years)

list

In [19]:
def round_year (years):
    for i in years:
        round(i, -1)
    return year
    

In [20]:
round_years = map(round_year, years)

In [ ]:
import numpy as np
years_change = [500,77,5,0]
df2.Year.replace(years_change, np.NaN)
df2.Year.replace('<NA>',np.NaN)

In [13]:
def clean_year (years):
    for i in years:
        if i < 1543:
            i == "NaN"
    return years

In [14]:
clean_year(year)

NameError: name 'year' is not defined